In [1]:
import tensorflow as tf
import numpy as np
import time
print(tf.__version__)

1.10.0


In [2]:
import data_reader
SemData = data_reader.read_data_sets("data")

TRAIN_FILE.txt existed
TEST_FILE.txt existed


In [3]:
print(SemData.train.sentences.shape)
print(SemData.test.sentences.shape)

(8000, 97, 300)
(2717, 97, 300)


In [4]:
learning_rate = 2e-4
batch_size = 100
display_step = 200
test_size = SemData.test.labels.shape[0]

n_input = 300
n_steps = 97
n_hidden = 100
n_classes = 19

In [5]:
X = tf.placeholder(tf.float32, [None, n_steps, n_input])
Y = tf.placeholder(tf.float32, [None, n_classes])

In [8]:
def model(x):
    x = tf.transpose(x, [1, 0, 2])
    x = tf.unstack(x, n_steps)
    
    W_fw = tf.Variable(tf.truncated_normal([n_input, n_hidden], stddev=0.1))
    U_fw = tf.Variable(tf.truncated_normal([n_hidden, n_hidden], stddev=0.1))
    b_fw = tf.Variable(tf.truncated_normal([n_hidden], stddev=0.1))

    W_bw = tf.Variable(tf.truncated_normal([n_input, n_hidden], stddev=0.1))
    U_bw = tf.Variable(tf.truncated_normal([n_hidden, n_hidden], stddev=0.1))
    b_bw = tf.Variable(tf.truncated_normal([n_hidden], stddev=0.1))

    h0_fw = tf.Variable(tf.truncated_normal([1, n_hidden], stddev=0.1))
    h0_bw = tf.Variable(tf.truncated_normal([1, n_hidden], stddev=0.1))

    h_fw = []
    h_bw = []
    h = []
    
    d_conv1 = 6
    d_conv2 = 12
    d_fc1 = 60
    d_fc2 = 19
    
    W_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, d_conv1], stddev=0.1))
    B_conv1 = tf.Variable(tf.constant(0.1, tf.float32, [d_conv1]))
    W_conv2 = tf.Variable(tf.truncated_normal([5, 5, d_conv1, d_conv2], stddev=0.1))
    B_conv2 = tf.Variable(tf.constant(0.1, tf.float32, [d_conv2]))
    
    W_fc1 = tf.Variable(tf.truncated_normal([n_steps*n_hidden*d_conv2, d_fc1], stddev=0.1))
    B_fc1 = tf.Variable(tf.constant(0.1, tf.float32, [d_fc1]))
    W_fc2 = tf.Variable(tf.truncated_normal([d_fc1, d_fc2], stddev=0.1))
    B_fc2 = tf.Variable(tf.constant(0.1, tf.float32, [d_fc2]))
    
    for i in range(n_steps):
        if i == 0:
            h_fw.append(tf.tanh(tf.matmul(x[0], W_fw) + tf.matmul(h0_fw, U_fw) + b_fw))
            h_bw.append(tf.tanh(tf.matmul(x[n_steps-1], W_bw) + tf.matmul(h0_bw, U_bw) + b_bw))
        else:
            h_fw.append(tf.tanh(tf.matmul(x[i], W_fw) + tf.matmul(h_fw[i-1], U_fw) + b_fw))
            h_bw.append(tf.tanh(tf.matmul(x[n_steps-1-i], W_bw) + tf.matmul(h_fw[i-1], U_bw) + b_bw))
    
    for i in range(n_steps):
        h.append(h_fw[i] + h_bw[n_steps-1-i])
    
    h = tf.stack(h)
    h = tf.transpose(h, [1, 0, 2])
    h = tf.reshape(h, [-1, n_steps, n_hidden, 1])
    
    stride = 1
    h_conv1 = tf.nn.relu(tf.nn.conv2d(h, W_conv1, strides=[1, stride, stride, 1], padding='SAME') + B_conv1)
    h_conv2 = tf.nn.relu(tf.nn.conv2d(h_conv1, W_conv2, strides=[1, stride, stride, 1], padding='SAME') + B_conv2)
    
    hh = tf.reshape(h_conv2, shape=[-1, n_steps*n_hidden*d_conv2])
    
    h_fc1 = tf.nn.relu(tf.matmul(hh, W_fc1) + B_fc1)
    hh_fc1 = tf.nn.dropout(h_fc1, 0.5)
    h_fc2 = tf.matmul(hh_fc1, W_fc2) + B_fc2
    pred = tf.nn.softmax(h_fc2)
    
    return pred

In [9]:
Y_pred = model(X)

In [10]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=Y_pred, labels=Y))

correct_prediction = tf.equal(tf.argmax(Y_pred, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [12]:
start = time.clock()
test_X, test_Y = SemData.test.next_batch(300)
test_X = test_X.reshape([300, n_steps, n_input])
for i in range(6000):
    batch_X, batch_Y = SemData.train.next_batch(batch_size)
    batch_X = batch_X.reshape([batch_size, n_steps, n_input])
    if i % display_step == 0:
        train_accuracy = accuracy.eval(session=sess, feed_dict={X:batch_X, Y:batch_Y})
        test_accuracy = accuracy.eval(session=sess, feed_dict={X:test_X, Y:test_Y})
        print("step %5d, train_accuracy = %.4g %% test_accuracy = %.4g %%" % (i, train_accuracy*100, test_accuracy*100))
    train_step.run(session=sess, feed_dict={X:batch_X, Y:batch_Y})
print("------------------------------------")
print("training time: ", time.clock()-start, " s")
test_accuracy = accuracy.eval(session=sess, feed_dict={X:test_X, Y:test_Y})
print("final accuracy = %.4g %%" % (test_accuracy*100))

step     0, train_accuracy = 7 % test_accuracy = 6.667 %
step   200, train_accuracy = 24 % test_accuracy = 17.67 %
step   400, train_accuracy = 14 % test_accuracy = 17.33 %
step   600, train_accuracy = 24 % test_accuracy = 17.67 %
step   800, train_accuracy = 16 % test_accuracy = 17 %
step  1000, train_accuracy = 24 % test_accuracy = 17.33 %
step  1200, train_accuracy = 14 % test_accuracy = 18.33 %


InternalError: Blas GEMM launch failed : a.shape=(100, 300), b.shape=(100, 100), m=300, n=100, k=100
	 [[Node: gradients/MatMul_568_grad/MatMul_1 = MatMul[T=DT_FLOAT, transpose_a=true, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](unstack_1:45, gradients/add_665_grad/Sum)]]

Caused by op 'gradients/MatMul_568_grad/MatMul_1', defined at:
  File "C:\Users\chuzi\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\chuzi\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\chuzi\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\chuzi\Anaconda3\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\chuzi\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\kernelapp.py", line 505, in st